In [92]:
import sqlite3

In [93]:
connect = sqlite3.connect('database.sqlite')
cur = connect.cursor()

In [94]:
!pip install pandas

# Task 3.1

In [95]:
import re
import sqlite3
from datetime import datetime

#Giving words categories of Tier1, Tier2, Tier3 as in the rules
TIER1_WORDS =['tier1badword', 'arsehole', 'asshole', 'beaner',
             'bitch', 'chink', 'clit', 'cock', 'cocksucker',
             'coon', 'cum', 'cumshot', 'cunt', 'dick', 'dickhead',
             'dildo', 'douche', 'dyke', 'fag', 'faggot', 'felch',
             'feltch', 'fuck', 'fudgepacker', 'gangbang', 'gook', 'heeb',
             'hentai', 'honkey', 'homo', 'jizz', 'kike', 'kyke', 'milf',
             'molest', 'motherfucker', 'muff', 'nazi', 'negro', 'nigga',
             'nigger', 'paki', 'pedophile', 'penis', 'piss', 'prick', 'pussy',
             'rape', 'rapist', 'retard', 'shemale', 'shit', 'shota', 'skank',
             'slut', 'smegma', 'spic', 'tits', 'titties', 'tranny', 'twat', 'vagina',
             'wetback', 'whore', 'yid']
TIER2_PHRASES =['tier2badword', '100% free', '100% satisfied', 'act now',
               'additional income', 'apply now', 'as seen on', 'be your own boss',
               'being ripped off', 'buy direct', 'buy now', 'cancel at any time',
               'cash bonus', 'cialis', 'claim your prize', 'clearance', 'click below',
               'click here', 'click here to win', 'compare rates',
               'congratulations you won', 'crypto giveaway', 'deal of the day',
               'double your income', 'earn extra cash', 'earn money', 'eliminate debt',
               'exclusive deal', 'expect to earn', 'fantastic deal', 'financial freedom',
               'for free', 'free access', 'free gift', 'free investment',
               'free money', 'free preview', 'free trial', 'get it now',
               'get paid', 'get rich quick', 'guaranteed winner',
               'important information regarding', 'income from home', 'increase sales',
               'instant winner', 'investment opportunity', 'join millions', 'limited time offer',
               'lowest price', 'make money fast', 'male enhancement', 'meet singles',
               'money back guarantee', 'multi-level marketing', 'no cost',
               'no credit check', 'no experience', 'no fees', 'no hidden costs',
               'no obligation', 'not a scam', 'one time offer', 'online pharmacy',
               'opportunity', 'order now', 'order today', 'please read', 'prize notification',
               'promise you', 'risk-free', 'satisfaction guaranteed', 'save big money', 'stop snoring',
               'subscribe now', "this isn't a scam", "this isn't spam", 'unlimited', 'urgent', 'viagra',
               'visit our website', 'weight loss', 'what are you waiting for', 'while supplies last',
               'winner chosen', 'work from home', 'you are a winner', 'you have been selected']
TIER3_WORDS =['tier3badword', 'arse', 'arsehat', 'ass', 'asshat', 'balls', 'bastard',
             'bellend', 'berk', 'bish', 'bloody', 'bollocks', 'boner', 'boobs', 'bugger',
             'bullcrap', 'bullshit', 'choad', 'clown', 'cockwomble', 'crap', 'crikey', 'damn',
             'damnit', 'dipshit', 'doofus', 'douche', 'douchebag', 'dumbass', 'effing', 'fart',
             'feck', 'flaming', 'frick', 'git', 'goddamn', 'gosh', 'hell', 'holy shit', 'horseshit',
             'jackass', 'jerk', 'jesus christ', 'jfc', 'knob', 'lmao', 'lmfao', 'minge', 'moron', 'munter',
             'naff', 'ninny', 'pillock', 'piss', 'pissed', 'pissed off', 'plonker', 'poon', 'poop', 'prat',
             'schlong', 'screw you', 'shag', 'shithead', 'shitting', 'sod off', 'son of a bitch', 'spaz',
             'turd', 'twatwaffle', 'wang', 'wank', 'wanker']

#Moderation
def moderate_content(text):
    if not text or not isinstance(text, str):
        return '', 0.0

    textLower = text.lower()

    #Stage 1.1
    for word in TIER1_WORDS:
        if re.search(rf'\b{re.escape(word)}\b', textLower):
            return '[content removed due to severe violation]', 5.0

    for phrase in TIER2_PHRASES:
        if phrase in textLower:
            return '[content removed due to spam/scam policy]', 5.0

    #Stage 1.2
    cleanedText = text
    contentScore = 0.0

    # Tier 3 words (replace and +2 each score)
    for word in TIER3_WORDS:
        pattern = rf'\b{re.escape(word)}\b'
        matches = re.findall(pattern, cleanedText, flags=re.IGNORECASE)
        if matches:
            contentScore += 2.0 * len(matches)
            cleanedText = re.sub(pattern, '*' * len(word), cleanedText, flags=re.IGNORECASE)

    # External links (replace and +2 each score)
    urlPattern = r'http[s]?://[^\s]+'
    if re.search(urlPattern, cleanedText):
        cleanedText = re.sub(urlPattern, '[link removed]', cleanedText)
        contentScore += 2.0

    # Excessive capitalization (+0.5 score)
    letters = [c for c in text if c.isalpha()]
    if len(letters) > 15:
        upperRatio = sum(1 for c in letters if c.isupper()) / len(letters)
        if upperRatio > 0.7:
            contentScore += 0.5

    return cleanedText, min(contentScore, 5.0)

# Risk Score Helpers (Part 2)
def calcAccountAge(createdAt):
    try:
        createdDate = datetime.strptime(createdAt, '%Y-%m-%d %H:%M:%S')
        return (datetime.now() - createdDate).days
    except:
        return 999

def calcPostRiskScore(content, createdAt):
    cleaned, baseScore = moderate_content(content)
    age = calcAccountAge(createdAt)
    return min(baseScore * 1.5, 5.0) if age < 7 else baseScore

def calcUserRiskScore(profile, posts, comments, createdAt):
    _, profileScore = moderate_content(profile or '')
    postScores = [moderate_content(p)[1] for p in posts] if posts else [0.0]
    commentScores = [moderate_content(c)[1] for c in comments] if comments else [0.0]
    avgPost = sum(postScores) / len(postScores)
    avgComment = sum(commentScores) / len(commentScores)
    contentRisk = (profileScore * 1) + (avgPost * 3) + (avgComment * 1)

    age = calcAccountAge(createdAt)
    if age < 7:
        userRisk = contentRisk * 1.5
    elif age < 30:
        userRisk = contentRisk * 1.2
    else:
        userRisk = contentRisk

    return min(userRisk, 5.0)

#Risk levels
def getRiskLevel(score):
    if score >= 5.0: return 'HIGH'
    if score >= 3.0: return 'MEDIUM'
    if score >= 1.0: return 'LOW'
    return 'NONE'

#Additional (Harrasement and Bullying detection )
def detectBullying(text):
    if not text or not isinstance(text, str):
        return False, 0.0, []

    textLower = text.lower()
    reasons = []
    bullyingScore = 0.0

    #bullying or harassment keywords
    insultWords = [
        'stupid', 'idiot', 'dumb', 'useless', 'pathetic', 'worthless',
        'loser', 'trash', 'jerk', 'moron', 'hate you', 'go away',
        'kill yourself', 'nobody likes you', 'ugly', 'fat', 'disgusting',
        'freak', 'weirdo', 'failure', 'crybaby', 'clown', 'embarrassing'
    ]

    # Only trigger when insult words appear directly
    safePhrases = [
        "don't be", "not be", "avoid being", "stop being",
        "called", "say", "said", "telling", "learning"
    ]
    if any(phrase in textLower for phrase in safePhrases):
        return False, 0.0, []

    # Count bullying words
    for word in insultWords:
        if re.search(rf'\b{re.escape(word)}\b', textLower):
            bullyingScore += 2.0
            reasons.append(f"Detected bullying word: '{word}'")

    # At least one strong insult
    if bullyingScore >= 2.0:
        isBullying = True
    else:
        isBullying = False
        bullyingScore = 0.0
        reasons = []

    return isBullying, min(bullyingScore, 5.0), reasons


# Making a report overall and printing
def analyzeDatabase(dbPath='database.sqlite'):
    conn = sqlite3.connect(dbPath)
    cur = conn.cursor()

    print('=' * 70)
    print('CONTENT MODERATION SUMMARY')
    print('=' * 70)

    #Posts
    cur.execute("""
        SELECT p.id, p.content, u.created_at, p.user_id
        FROM posts p JOIN users u ON p.user_id = u.id
    """)
    posts = cur.fetchall()
    print('\nPOSTS WITH VIOLATIONS:\n' + '-' * 70)
    for pid, content, createdAt, userId in posts:
        cleaned, score = moderate_content(content)
        risk = calcPostRiskScore(content, createdAt)
        if risk > 0:
            print(f'Post {pid} (User {userId}) → {getRiskLevel(risk)} | Score: {risk:.1f}')
            print(f'Original: {content[:80]}')
            print(f'Cleaned : {cleaned[:80]}\n')

    #Comments
    cur.execute("""
        SELECT c.id, c.content, u.created_at, c.user_id
        FROM comments c JOIN users u ON c.user_id = u.id
    """)
    comments = cur.fetchall()
    print('\nCOMMENTS WITH VIOLATIONS:\n' + '-' * 70)
    for cid, content, createdAt, userId in comments:
        cleaned, score = moderate_content(content)
        risk = calcPostRiskScore(content, createdAt)
        if risk > 0:
            print(f'Comment {cid} (User {userId}) → {getRiskLevel(risk)} | Score: {risk:.1f}')
            print(f'Original: {content[:80]}')
            print(f'Cleaned : {cleaned[:80]}\n')

    #Users
    cur.execute("SELECT id, username, profile, created_at FROM users")
    users = cur.fetchall()
    print('\nUSER RISK & BULLYING CHECKS:\n' + '-' * 70)
    for uid, username, profile, createdAt in users:
        cur.execute("SELECT content FROM posts WHERE user_id = ?", (uid,))
        userPosts = [p[0] for p in cur.fetchall()]
        cur.execute("SELECT content FROM comments WHERE user_id = ?", (uid,))
        userComments = [c[0] for c in cur.fetchall()]

        bullyFlag, bullyScore, reasons = detectBullying(profile)
        bullyingComments = []
        for c in userComments:
            flag, _, _ = detectBullying(c)
            if flag:
                bullyingComments.append(c[:80])

        userRisk = calcUserRiskScore(profile, userPosts, userComments, createdAt)

        if userRisk > 0 or bullyFlag or bullyingComments:
            print(f'User {uid} @{username} → {getRiskLevel(userRisk)} | Score: {userRisk:.1f}')
            if bullyFlag:
                print(f'  Bullying/Harassment risk {bullyScore:.1f} | Reasons: {", ".join(reasons)}')
            if bullyingComments:
                print(f'  Comments showing bullying or harassment:')
                for com in bullyingComments[:3]:
                    print(f'     "{com}"')
            if profile:
                print(f'  Profile: {profile[:80]}\n')

    conn.close()
    print('=' * 70)
    print('Complete.\n')

if __name__ == '__main__':
    analyzeDatabase('database.sqlite')


CONTENT MODERATION SUMMARY

POSTS WITH VIOLATIONS:
----------------------------------------------------------------------
Post 1750 (User 18) → HIGH | Score: 5.0
Original: My cat just knocked over my coffee again. I swear, one more time and I’ll lose m
Cleaned : [content removed due to severe violation]

Post 1756 (User 19) → MEDIUM | Score: 4.0
Original: Fucking hell, another day of dealing with idiots at work. Why do people have to 
Cleaned : Fucking ****, another day of dealing with idiots at work. Why do people have to 

Post 1771 (User 22) → MEDIUM | Score: 4.0
Original: Why can't people mind their own damn business instead of pushing their vegan cra
Cleaned : Why can't people mind their own **** business instead of pushing their vegan ***

Post 1772 (User 22) → HIGH | Score: 5.0
Original: Who gives a shit about another fucking park? Waste of money if you ask me. #Rant
Cleaned : [content removed due to severe violation]

Post 1773 (User 22) → LOW | Score: 2.0
Original: Some dumb a

# Task 3.2

In [124]:
from datetime import datetime

def user_risk_analysis(userId, dbPath='database.sqlite'):
    cur.execute("SELECT username, profile, created_at FROM users WHERE id = ?", (userId,))
    row = cur.fetchone()
    if not row:
        return None
    username, profile, createdAt = row

    cur.execute("SELECT content FROM posts WHERE user_id = ?", (userId,))
    posts = [p[0] for p in cur.fetchall()]
    cur.execute("SELECT content FROM comments WHERE user_id = ?", (userId,))
    comments = [c[0] for c in cur.fetchall()]

    _, profileScore = moderate_content(profile or "")
    postScores = [moderate_content(p)[1] for p in posts] if posts else [0.0]
    commentScores = [moderate_content(c)[1] for c in comments] if comments else [0.0]

    avgPost = sum(postScores) / len(postScores)
    avgComment = sum(commentScores) / len(commentScores)
    contentRisk = (profileScore * 1) + (avgPost * 3) + (avgComment * 1)

    try:
        createdDate = datetime.strptime(createdAt, "%Y-%m-%d %H:%M:%S")
        ageDays = (datetime.now() - createdDate).days
    except:
        ageDays = 999

    if ageDays < 7:
        userRisk = contentRisk * 1.5
    elif ageDays < 30:
        userRisk = contentRisk * 1.2
    else:
        userRisk = contentRisk

    totalPosts = len(posts) + len(comments)
    postingIntensityRisk = 0.0
    if ageDays < 7:
        if totalPosts > 15:
            postingIntensityRisk = 1.0
        elif totalPosts > 8:
            postingIntensityRisk = 0.5

    finalRisk = min(round(userRisk + postingIntensityRisk, 2), 5.0)

    if finalRisk >= 5.0:
        riskLevel = "HIGH"
    elif finalRisk >= 3.0:
        riskLevel = "MEDIUM"
    elif finalRisk >= 1.0:
        riskLevel = "LOW"
    else:
        riskLevel = "NONE"

    conn.close()
    return {
        "userId": userId,
        "username": username,
        "postingIntensityRisk": postingIntensityRisk,
        "finalRisk": finalRisk,
        "riskLevel": riskLevel
    }

def top5RiskyUsers(dbPath='database.sqlite'):
    conn = sqlite3.connect(dbPath)
    cur = conn.cursor()
    cur.execute("SELECT id FROM users")
    userIds = [r[0] for r in cur.fetchall()]
    conn.close()

    scores = []
    for uid in userIds:
        data = user_risk_analysis(uid, dbPath)
        if data:
            scores.append(data)

    scores.sort(key=lambda x: (-x["finalRisk"], x["username"]))
    top5 = scores[:5]

    print("TOP 5 HIGHEST RISK USERS")
    for i, u in enumerate(top5, 1):
        print(f"\n#{i}  User ID: {u['userId']}  |  @{u['username']}")
        print(f"Posting Intensity Risk: {u['postingIntensityRisk']:.2f}")
        print(f"Final Risk Score: {u['finalRisk']:.2f}")
        print(f"Risk Level: {u['riskLevel']}")

    return top5

if __name__ == "__main__":
    top5RiskyUsers('database.sqlite')


TOP 5 HIGHEST RISK USERS

#1  User ID: 66  |  @BlackPearl
Posting Intensity Risk: 0.00
Final Risk Score: 5.00
Risk Level: HIGH

#2  User ID: 11  |  @ChilliPepper
Posting Intensity Risk: 0.00
Final Risk Score: 5.00
Risk Level: HIGH

#3  User ID: 69  |  @RedRose
Posting Intensity Risk: 0.00
Final Risk Score: 5.00
Risk Level: HIGH

#4  User ID: 68  |  @SilverFox
Posting Intensity Risk: 0.00
Final Risk Score: 5.00
Risk Level: HIGH

#5  User ID: 145  |  @ostrich_oliver
Posting Intensity Risk: 0.00
Final Risk Score: 4.02
Risk Level: MEDIUM


# Task 3.3

In [136]:
import sqlite3

def recommend(userId, dbPath='database.sqlite'):
    print(f"Generating Recommendations for User {userId}")
    conn = sqlite3.connect(dbPath)
    cur = conn.cursor()
    # Get posts liked by user
    try:
        cur.execute("""
            SELECT DISTINCT post_id
            FROM reaction
            WHERE user_id = ? AND (type='like' OR type='positive' OR value=1)
        """, (userId,))
        likedPosts = [row[0] for row in cur.fetchall()]
    except sqlite3.OperationalError:
        likedPosts = []

    # Get authors of liked posts
    likedAuthors = []
    if likedPosts:
        qMarks = ",".join("?" * len(likedPosts))
        cur.execute(f"SELECT DISTINCT user_id FROM posts WHERE id IN ({qMarks})", likedPosts)
        likedAuthors = [row[0] for row in cur.fetchall()]

    # Get followed users
    try:
        cur.execute("SELECT followed_id FROM follows WHERE follower_id = ?", (userId,))
        followedUsers = [row[0] for row in cur.fetchall()]
    except sqlite3.OperationalError:
        followedUsers = []
        print("'follows' table not found. Skipping follow-based recommendations...")

    # Combine all related users
    relatedUsers = list(set(likedAuthors + followedUsers))


    print(f"Followed Users: {followedUsers or 'None'}")
    print(f"Related Users: {relatedUsers or 'None'}\n")

    # Recommend recent posts
    if relatedUsers:
        qMarks = ",".join("?" * len(relatedUsers))
        cur.execute(f"""
            SELECT id, content
            FROM posts
            WHERE user_id IN ({qMarks}) AND user_id != ?
            ORDER BY created_at DESC
            LIMIT 5
        """, (*relatedUsers, userId))
    else:
        print("No liked/followed users found, showing 5 latest posts instead.\n")
        cur.execute("""
            SELECT id, content
            FROM posts
            WHERE user_id != ?
            ORDER BY created_at DESC
            LIMIT 5
        """, (userId,))

    posts = cur.fetchall()

    if posts:
        print("Recommended Posts:\n" + "-" * 60)
        for pid, content in posts:
            print(f"Post ID {pid}: {content[:80]}")
    else:
        print("No posts found in database to recommend.")
    conn.close()
    return posts


# Example
if __name__ == "__main__":
    # Fetch 5 user IDs and print recommendations for them
    conn = sqlite3.connect("database.sqlite")
    cur = conn.cursor()
    cur.execute("SELECT id FROM users ORDER BY id LIMIT 5")
    userIds = [r[0] for r in cur.fetchall()]
    conn.close()

    for uid in userIds:
        recommend(uid, "database.sqlite")
        print("\n" + "="*60 + "\n")


Generating Recommendations for User 1
Followed Users: [6, 8, 10, 12, 16, 17, 22, 24, 26, 27, 28, 36, 37, 40, 42, 54, 56, 57, 64, 71, 74, 80, 81, 89, 96, 100, 101, 109, 110, 111, 112, 113, 116, 120, 121, 129, 130, 132, 133, 137, 139, 143, 149, 150, 506]
Related Users: [129, 130, 132, 133, 6, 8, 137, 10, 139, 12, 143, 16, 17, 149, 22, 150, 24, 26, 27, 28, 36, 37, 40, 42, 54, 56, 57, 64, 71, 74, 80, 81, 89, 96, 100, 101, 109, 110, 111, 112, 113, 116, 120, 121, 506]

Recommended Posts:
------------------------------------------------------------
Post ID 2781: Feeling grateful for the small moments today. Sometimes, a quiet afternoon with 
Post ID 2830: Just heard about a local art exhibition featuring mental health themes. Can't wa
Post ID 2843: Can’t stop thinking about last night's concert. The energy was electric! #LiveMu
Post ID 2780: Just learned about a new policy supporting wildlife conservation in Australia. I
Post ID 2842: Finally learned how to play that tricky guitar riff. Feels